In [1]:
import pandas as pd
import numpy as np
import warnings
from typing import List, Dict, Tuple

warnings.filterwarnings('ignore')

## 1. Configuración y Carga de Datos

In [2]:
RUTA_DATOS_ENTRADA = '/Users/tabotavin/Desktop/MNA-MLOps-Proyecto-Equipo03/data/turkish_music_emotion_modified.csv'
RUTA_DATOS_SALIDA = '/Users/tabotavin/Desktop/MNA-MLOps-Proyecto-Equipo03/data/turkish_music_emotion_cleaned.csv'

df = pd.read_csv(RUTA_DATOS_ENTRADA)
df.shape

(408, 52)

## 2. Funciones Modulares de Limpieza

### 2.1. Función de Análisis de Calidad

In [3]:
def identificar_problemas_calidad(df: pd.DataFrame) -> Dict[str, any]:
    """
    Identifica problemas de calidad en el dataset.
    
    Args:
        df: DataFrame a analizar
        
    Returns:
        Diccionario con resumen de problemas identificados
    """
    problemas = {
        'columnas_extra': [],
        'valores_invalidos': {},
        'columnas_con_nulos': [],
        'filas_con_problemas': 0
    }
    
    # Identificar columnas que parecen no pertenecer al dataset
    for col in df.columns:
        # Buscar columnas con nombres poco estándar o mezcla de tipos
        if 'mixed' in col.lower() or 'col' in col.lower():
            problemas['columnas_extra'].append(col)
    
    # Identificar valores inválidos comunes
    valores_invalidos_buscar = ['NULL', 'NaN', 'nan', 'error', 'invalid', 'bad', 
                                'unknown', '?', 'ERROR', 'N/A', '', ' ']
    
    for col in df.columns:
        if df[col].dtype == 'object':
            count = 0
            for val in valores_invalidos_buscar:
                count += (df[col] == val).sum()
                count += df[col].str.strip().eq('').sum() if hasattr(df[col], 'str') else 0
            
            if count > 0:
                problemas['valores_invalidos'][col] = count
    
    # Identificar columnas con valores nulos
    nulos = df.isnull().sum()
    problemas['columnas_con_nulos'] = nulos[nulos > 0].index.tolist()
    
    return problemas

### 2.2. Funciones de Limpieza de Valores

In [4]:
def limpiar_valores_invalidos(df: pd.DataFrame, columnas_excluir: List[str] = None) -> pd.DataFrame:
    """
    Limpia valores inválidos convirtiéndolos a NaN.
    """
    if columnas_excluir is None:
        columnas_excluir = []
    
    valores_invalidos = ['NULL', 'NaN', 'nan', 'error', 'invalid', 'bad', 'unknown', 
                        '?', 'ERROR', 'N/A', '', ' ']
    
    df_clean = df.copy()
    total_convertidos = 0
    
    for col in df_clean.columns:
        if col in columnas_excluir:
            continue
            
        if df_clean[col].dtype == 'object':
            mask = df_clean[col].isin(valores_invalidos)
            if hasattr(df_clean[col], 'str'):
                mask |= df_clean[col].str.strip().eq('')
            
            conversiones = mask.sum()
            if conversiones > 0:
                df_clean.loc[mask, col] = np.nan
                total_convertidos += conversiones
    
    return df_clean

In [5]:
def eliminar_columnas_innecesarias(df: pd.DataFrame, columnas: List[str]) -> pd.DataFrame:
    """
    Elimina columnas innecesarias del dataset.
    """
    columnas_existentes = [col for col in columnas if col in df.columns]
    if columnas_existentes:
        df = df.drop(columns=columnas_existentes)
    return df

### 2.3. Funciones de Gestión de Columnas

In [6]:
def normalizar_variable_categorica(df: pd.DataFrame, columna: str) -> pd.DataFrame:
    """
    Normaliza una variable categórica.
    """
    if columna in df.columns:
        df[columna] = df[columna].astype(str).str.strip().str.lower()
    return df

In [7]:
def convertir_a_tipos_correctos(df: pd.DataFrame, columnas_excluir: List[str] = None) -> pd.DataFrame:
    """
    Convierte columnas a sus tipos de datos correctos.
    """
    if columnas_excluir is None:
        columnas_excluir = []
    
    df_typed = df.copy()
    
    for col in df_typed.columns:
        if col not in columnas_excluir:
            df_typed[col] = pd.to_numeric(df_typed[col], errors='coerce')
    
    return df_typed

### 2.4. Funciones de Conversión de Tipos

In [8]:
def eliminar_filas_con_nulos_criticos(df: pd.DataFrame, columna_critica: str) -> pd.DataFrame:
    """
    Elimina filas que tienen valores nulos o inválidos en columnas críticas.
    """
    filas_antes = len(df)
    
    df_clean = df.dropna(subset=[columna_critica])
    
    if df_clean[columna_critica].dtype == 'object':
        df_clean = df_clean[~df_clean[columna_critica].astype(str).str.lower().isin(['nan', 'none', ''])]
    
    filas_eliminadas = filas_antes - len(df_clean)
    print(f"Filas eliminadas: {filas_eliminadas}")
    
    return df_clean

### 2.5. Funciones de Gestión de Filas

In [9]:
def imputar_valores_faltantes(df: pd.DataFrame, metodo: str = 'mediana', 
                              columnas_excluir: List[str] = None) -> pd.DataFrame:
    """
    Imputa valores faltantes usando método estadístico.
    """
    if columnas_excluir is None:
        columnas_excluir = []
    
    df_imputed = df.copy()
    
    columnas_numericas = df_imputed.select_dtypes(include=[np.number]).columns
    columnas_a_imputar = [col for col in columnas_numericas if col not in columnas_excluir]
    
    valores_imputados = 0
    for col in columnas_a_imputar:
        nulos_col = df_imputed[col].isnull().sum()
        if nulos_col > 0:
            if metodo == 'mediana':
                valor_imputacion = df_imputed[col].median()
            elif metodo == 'media':
                valor_imputacion = df_imputed[col].mean()
            else:
                valor_imputacion = df_imputed[col].mode()[0]
            
            df_imputed[col].fillna(valor_imputacion, inplace=True)
            valores_imputados += nulos_col
    
    print(f"Valores imputados ({metodo}): {valores_imputados}")
    
    return df_imputed

### 2.6. Funciones de Imputación y Reportes

In [10]:
def generar_reporte_limpieza(df_inicial: pd.DataFrame, df_final: pd.DataFrame) -> Dict[str, any]:
    """
    Genera reporte del proceso de limpieza.
    
    Args:
        df_inicial: DataFrame antes de limpieza
        df_final: DataFrame después de limpieza
        
    Returns:
        Diccionario con métricas del proceso
    """
    reporte = {
        'filas_inicial': len(df_inicial),
        'filas_final': len(df_final),
        'filas_eliminadas': len(df_inicial) - len(df_final),
        'columnas_inicial': len(df_inicial.columns),
        'columnas_final': len(df_final.columns),
        'columnas_eliminadas': len(df_inicial.columns) - len(df_final.columns),
        'nulos_inicial': df_inicial.isnull().sum().sum(),
        'nulos_final': df_final.isnull().sum().sum(),
        'tasa_retencion': (len(df_final) / len(df_inicial) * 100) if len(df_inicial) > 0 else 0
    }
    
    return reporte

## 3. Análisis Inicial de Datos Corruptos

In [11]:
df_original = df.copy()

problemas = identificar_problemas_calidad(df)

print(f"Columnas extra: {problemas['columnas_extra']}")
print(f"Columnas con valores inválidos: {len(problemas['valores_invalidos'])}")
print(f"Columnas con nulos: {len(problemas['columnas_con_nulos'])}")

Columnas extra: ['mixed_type_col']
Columnas con valores inválidos: 35
Columnas con nulos: 52


## 4. Ejecución del Pipeline de Limpieza

In [12]:
# Paso 1: Eliminar columnas innecesarias
if problemas['columnas_extra']:
    df = eliminar_columnas_innecesarias(df, problemas['columnas_extra'])
    print(f"Columnas eliminadas: {problemas['columnas_extra']}")

df.shape

Columnas eliminadas: ['mixed_type_col']


(408, 51)

In [13]:
# Paso 2: Normalizar variable objetivo
df = normalizar_variable_categorica(df, 'Class')
df['Class'].unique()

array(['relax', 'happy', 'sad', 'angry', 'nan'], dtype=object)

In [14]:
# Paso 3: Limpiar valores inválidos
df = limpiar_valores_invalidos(df, columnas_excluir=['Class'])
df.isnull().sum().sum()

np.int64(295)

In [15]:
# Paso 4: Convertir a tipos de datos correctos
df = convertir_a_tipos_correctos(df, columnas_excluir=['Class'])
df.dtypes.value_counts()

float64    50
object      1
Name: count, dtype: int64

In [16]:
# Paso 5: Eliminar filas con problemas críticos en variable objetivo
df = eliminar_filas_con_nulos_criticos(df, 'Class')
df.shape

Filas eliminadas: 5


(403, 51)

In [17]:
# Paso 6: Imputar valores faltantes
df = imputar_valores_faltantes(df, metodo='mediana', columnas_excluir=['Class'])
df.isnull().sum().sum()

Valores imputados (mediana): 298


np.int64(0)

## 5. Validación de Resultados

In [18]:
print(f"Dimensiones: {df.shape}")
print(f"Valores nulos: {df.isnull().sum().sum()}")
print(f"Valores duplicados: {df.duplicated().sum()}")

print(f"\nVariable objetivo - Valores únicos: {df['Class'].nunique()}")
for clase, count in df['Class'].value_counts().items():
    print(f"  {clase}: {count} ({count/len(df)*100:.1f}%)")

Dimensiones: (403, 51)
Valores nulos: 0
Valores duplicados: 2

Variable objetivo - Valores únicos: 4
  relax: 102 (25.3%)
  happy: 102 (25.3%)
  sad: 102 (25.3%)
  angry: 97 (24.1%)


In [19]:
columnas_muestra = ['_RMSenergy_Mean', '_Tempo_Mean', '_Brightness_Mean', 
                    '_Spectralcentroid_Mean', '_Eventdensity_Mean']

columnas_disponibles = [col for col in columnas_muestra if col in df.columns]

if columnas_disponibles:
    stats_df = df[columnas_disponibles].describe().T
    stats_df = stats_df[['mean', 'std', 'min', 'max']].round(4)
    display(stats_df)

,mean,std,min,max
_RMSenergy_Mean,3.2873,47.5225,0.010,873.096
_Tempo_Mean,153.1517,601.4155,48.284,12177.714
_Brightness_Mean,5.9359,64.1229,0.053,852.419
_Spectralcentroid_Mean,4612.9601,21684.0952,606.524,318051.072
_Eventdensity_Mean,8.8927,67.5416,0.234,998.241


## 6. Exportación del Dataset Limpio

In [20]:
df.to_csv(RUTA_DATOS_SALIDA, index=False)